In [1]:
import numpy as np
import random
from higherorder.structures.structures import Grid, Graph, Structure
from higherorder.dynamics.model import Model
import higherorder.analysis.ca as simulations
from higherorder.dynamics.rules import *
from higherorder.dynamics.impacts import *
from higherorder.utils import *

Load randomly initialized grids

In [2]:
grids_dicts1 = load_init_grid_dicts(".\\data\\oneblob_grids.json", listify=True)
grids_dicts2 = load_init_grid_dicts(".\\data\\twoblob_grids.json", listify=True)
grids_dicts3 = load_init_grid_dicts(".\\data\\two_from_oneblob.json", listify=True)

Cast back to grid

In [3]:
height = 20
width = 20

grids1 = []
for i in range(len(grids_dicts1)):
    grids1.append(Grid(grids_dicts1[i], width=width, height=height))

grids3 = []
for i in range(len(grids_dicts3)):
    grids3.append(Grid(grids_dicts3[i], width=width, height=height))

This takes considerably longer to load than the other two sources (19000 grids vs 2x2400 grids)

In [ ]:
grids2 = []
for i in range(len(grids_dicts2)):
    grids2.append(Grid(grids_dicts2[i], width=width, height=height))

Run Game of Life simulation till repeating structure - better would be till repeating blobs (but also consider e.g. non stop till blobs "have a chance to meet" e.g. go in the same direction)

In [4]:
states1 = []
impact1 = []
for i in range(len(grids1)):
    sim = Model(grids1[i], dynamics_func=game_of_life,)
    sim.simulate_till_periodicity(store_impact=True, impact_function=game_of_life_impact)
    states1.append(sim.structure.get_entities_states())
    impact1.append(sim.impact)

In [ ]:
states3 = []
impact3 = []
for i in range(len(grids3)):
    sim = Model(grids3[i], dynamics_func=game_of_life,)
    sim.simulate_till_periodicity(store_impact=True, impact_function=game_of_life_impact)
    states3.append(sim.structure.get_entities_states())
    impact3.append(sim.impact)

In [ ]:
states2 = []
impact2 = []
for i in range(len(grids2)):
    sim = Model(grids2[i], dynamics_func=game_of_life,)
    sim.simulate_till_periodicity(store_impact=True, impact_function=game_of_life_impact)
    states2.append(sim.structure.get_entities_states())
    impact2.append(sim.impact)

Across all examples: pick 2 random neighboring nodes (or aggregate across such) which:
- did not impact each other
- did impact each other
- had turn off impact e.g.?
- had 0,0 all the time
- had 0,1 or 0,0 only
- had 0,1 and 1,0 too
- had 1,1

Mutual information is enough?
For not pairs, we would use HOI

Watch out for time delay. Need to consider: spatially moving entropy?

## Analysis of the 8 cells around a target cell 

All possible combinations

In [ ]:
def dec_to_bin(x, binary_length=9):
    return np.array([int(b) for b in format(x, f'0{binary_length}b')])

def bin_to_dec(bin, binary_length=9):
    return sum([pow(2,binary_length-j-1)*i for j,i in enumerate(bin)])

nodes = [(0,0), (0,1), (0,2), (1,0), (1,1), (1,2), (2,0), (2,1), (2,2)]
states_all = []
impacts_all = []
categories = {}

for i in range(512):
    binary_vals = dec_to_bin(i)
    initial_arr = np.zeros((3, 3))
    for j in range(len(nodes)):
        initial_arr[nodes[j]] = binary_vals[j]
    grid = Grid(initial_arr, width=3, height=3)
    sim = Model(grid, dynamics_func=game_of_life,)
    sim.simulation(steps=1, store_impact=True, impact_function=game_of_life_impact)
    states = sim.structure.get_time_slice(key_name="t_1", only_nonzero=False)
    states_all.append(states)
    impacts_all.append(sim.impact)
    
    around = (binary_vals).copy()
    around[4] = 0
    s = np.sum(around)
    if s not in categories.keys():
        categories[s] = []
    categories[s].append((binary_vals,states[(1,1)], sim.impact)) #TODO

In [91]:
time_arrays = {}
y_values = {}
for n in range(9):
    instances = len(categories[n])
    time_array = np.zeros((instances, 9))
    ys = np.zeros(instances)
    for j,instance in enumerate(categories[n]):
        for k in range(9):
            time_array[j][k] = instance[0][k]
        ys[j] = instance[1]
    time_arrays[n] = time_array
    y_values[n] = ys

Node may have impact on itself - implement

In [ ]:
from hoi.metrics import Oinfo, GradientOinfo
from hoi.simulation import simulate_hoi_gauss
import matplotlib.pyplot as plt

def compute_hoi_beh(x):
    """This function computes the HOI using the Oinfo."""
    model = Oinfo(x)
    hoi = model.fit(method="gc", minsize=3, maxsize=3)
    return hoi.squeeze()

def compute_hoi_enc(x, y):
    """This function computes the HOI using the Oinfo."""
    model = GradientOinfo(x, y)
    hoi = model.fit(method="gc", minsize=3, maxsize=3)
    return hoi.squeeze()

In [120]:
compute_hoi_enc(time_arrays[1], y_values[1])

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

array([        nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,         nan,         nan,
               nan,         nan,         nan,  1.1303711 ,  1.0745687 ,
        1.0252914 ,  1.0031595 ,  0.98238564,  0.96058655,  0.90230656,
        0.843091  ,  0.8106909 ,  0.77971077,  0.7465792 ,  0.70569324,
        0.6743994 ,  0.6444597 ,  0.61329174,  0.5897932 ,  0.539732  ,
        0.4842043 ,  0.48209476,  0.41862297,  0.35691833,  0.7773762 ,
        0.6811085 ,  0.6483011 ,  0.61636066,  0.5812874 ,  0.6193905 ,
        0.58688164,  0.55597496,  0.52381134,  0.44302845,  0.39468956,
        0.33874893,  0.33684063,  0.27319908,  0.20243168,  0.5445099 ,
        0.50988007,  0.47684097,  0.44213676,  0.3064804 ,  0.25

In [117]:
compute_hoi_enc(time_arrays[2], y_values[2])

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

array([ 0.3731699 ,  0.3656845 ,         nan,  0.35837555,  0.35298634,
        0.3492098 ,  0.34614754,  0.30877686,         nan,  0.29081535,
        0.28050232,  0.27330208,  0.26744747,         nan,  0.24810123,
        0.23389244,  0.22396946,  0.21587181,         nan,         nan,
               nan,         nan,  0.18430233,  0.17102146,  0.15992737,
        0.14556885,  0.13316536,  0.11512184,  0.26466846,         nan,
        0.24345398,  0.23511124,  0.22971725,  0.22560978,         nan,
        0.20192337,  0.19025993,  0.1826067 ,  0.17666626,         nan,
               nan,         nan,         nan,  0.13993073,  0.12931538,
        0.12085629,  0.10938549,  0.10002232,  0.08637428,         nan,
        0.1538353 ,  0.14197636,  0.13439941,  0.12864685,         nan,
               nan,         nan,         nan,  0.08931732,  0.07918262,
        0.07137108,  0.06192017,  0.0535078 ,  0.04179382,         nan,
               nan,         nan,         nan,  0.04473782,  0.03

In [118]:
compute_hoi_enc(time_arrays[3], y_values[3])

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

array([        nan,         nan,         nan,  0.9465202 ,         nan,
               nan,  0.1415298 ,         nan,         nan,  0.5197621 ,
               nan,         nan, -0.29539216,         nan,  0.35373697,
               nan,         nan, -0.46800327,  0.4485252 ,         nan,
               nan, -0.35894197,         nan,         nan, -0.57748115,
               nan, -0.5984609 , -0.6098037 ,  0.34148026,  0.42703533,
        0.28529263,  0.2745905 ,  0.2679882 ,  0.26206875,  0.33910656,
        0.16339684,  0.1498766 ,  0.14151287,  0.13400936,  0.28931427,
        0.28025913,  0.27476597,  0.2698984 ,  0.06668377,  0.05688381,
        0.04805851,  0.04120255,  0.03209877,  0.02162647,  0.2330246 ,
        0.05193424,  0.03810406,  0.02951527,  0.02176857,  0.19037437,
        0.17948818,  0.1729269 ,  0.16708279, -0.03341961, -0.04298687,
       -0.05176067, -0.05588818, -0.06492615, -0.07493973,  0.14113331,
        0.12922001,  0.12204647,  0.11566353, -0.08387566, -0.09

In [119]:
compute_hoi_enc(time_arrays[4], y_values[4])

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

    Copnorm and demean the data
Get list of multiplets


  0%|          |  0/1 [00:00<?,       ?it/s]

array([        nan,  0.9363835 ,         nan,         nan,  0.40497088,
        0.9021014 ,         nan,  0.5244877 ,         nan,         nan,
       -0.0325368 ,  0.4621721 ,         nan,         nan,         nan,
       -0.2074275 ,  0.28585202,         nan,         nan, -0.07989901,
        0.4169892 ,         nan, -0.29411027,  0.19806269,         nan,
        0.18360436,         nan,         nan,  0.28214836,  0.38551235,
        0.23414993,  0.22582245,  0.22037888,  0.21534538,  0.29978848,
        0.11304283,  0.10269451,  0.09595013,  0.08970547,  0.26142025,
        0.2545643 ,  0.2501917 ,  0.24619484,  0.03711319,  0.0294714 ,
        0.0223875 ,  0.01824474,  0.01099205,  0.00294876,  0.19422245,
        0.00376034, -0.00667095, -0.01345634, -0.01977444,  0.16207314,
        0.15354443,  0.14823246,  0.14339924, -0.06040287, -0.06784248,
       -0.07481861, -0.07711697, -0.08424091, -0.09192657,  0.11604404,
        0.10676575,  0.10103321,  0.09583378, -0.08910751, -0.09

TODO check separately measures of redundancy, and synergy

Connect it with impact? e.g. impact can cover synergy, opposite can cover redundancy possibly? Only select impacts going to the target cell

## Other analysis

In [ ]:
for i in range(height):
    for j in range(width):
        cell = (i, j)
        pass

Variables array creation - watch out to store the same nodes in the same columns across arrays

Then use HOI on random neighboring nodes, random nodes at distance d, then do the same with only "inside blob only" nodes (how?)

Impact- into simulation functions as option, for now only GoL impact implementation.